<a href="https://colab.research.google.com/github/karaage0703/colab-notebooks/blob/main/kindle_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kindleの蔵書分析

[Kindleの蔵書情報をKindleアプリのXMLをパースして可視化する方法](https://zenn.dev/karaage0703/articles/3a163290a4bc26)参照

## データのダウンロードと確認

`KindleSyncMetadataCache.xml`をアップロードします。

In [ ]:
from google.colab import files
uploaded = files.upload()

必要なライブラリをインポート

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import csv

[Kindle for PCのキャッシュファイルから持っているKindle本の一覧を作成する](https://note.com/nanigashi/n/n8a1e590d2ea3)と[高橋さんかずひとさんのブログ記事](https://kazuhito00.hatenablog.com/entry/2022/12/31/153847)を参考にcsvに変換

In [ ]:
#kindle App cashe file => kindle book list csv
import xml.etree.ElementTree as ET
import csv

input_file_str = "./KindleSyncMetadataCache.xml"
output_file_str = "./Kindle.csv"
header = ["ASIN", "title", "authors", "publisher",
           "publication_date", "purchase_date",
           "textbook_type", "cde_contenttype",
           "content_type", "origins"]
nary = [header]
tree = ET.parse(input_file_str)
root = tree.getroot()

for book_info in root[2]:
   ary = []
   for info in book_info:
       #authers publishers are nested
       if len(info) == 0:
           if info.text is not None:
               ary.append(info.text)
           else:
               ary.append('')
       else:
           info_list = [ s.text for s in info ]
           for index, value in enumerate(info_list):
               if value is None:
                   info_list[index] = ''
           ary.append(';'.join(info_list))
   nary.append(ary)

with open(output_file_str, 'w') as f:
   writer = csv.writer(f)
   writer.writerows(nary)

`Kindle.csv`をPandasで読み込み、一部を表示

In [ ]:
df = pd.read_csv('Kindle.csv', sep=',')
df.head()

## データの前処理

購入日をインデックスとしてdatetime形式に変換

In [ ]:
df.set_index('purchase_date', inplace=True)
df.index = pd.to_datetime(df.index, format='%Y/%m/%d')

df.head()

年・月・日の列を作る

In [ ]:
df['year'] = list(pd.Series(df.index).apply(lambda x: x.year))
df['month'] = list(pd.Series(df.index).apply(lambda x: x.month))
df['day'] = list(pd.Series(df.index).apply(lambda x: x.day))
df['day_name'] = list(pd.Series(df.index).apply(lambda x: x.day_name()))
df['weekday'] = list(pd.Series(df.index).apply(lambda x: x.weekday()))

df.head()

年・月・日・曜日毎のデータを作成。代表で年毎のデータを表示

In [ ]:
year_data = df.groupby(['year']).size()
month_data = df.groupby(['month']).size()
day_data =  df.groupby(['day']).size()
day_name_data =  df.groupby(['day_name']).size()
weekday_data = df.groupby(['weekday']).size()

target_year = 2022
month_data2 = df[df['year']==target_year].groupby(['month']).size()
day_data2 =  df[df['year']==target_year].groupby(['day']).size()
day_name_data2 =  df[df['year']==target_year].groupby(['day_name']).size()
weekday_data2 = df[df['year']==target_year].groupby(['weekday']).size()

year_data.head(15)

タイトルのカッコの中にある出版社情報や巻数情報を削除

In [ ]:
df['title_renamed'] = df['title'].str.replace(r'\s*\([^()]*\)','').str.replace(r'\s*\（[^()]*\）','').str.strip()

df.head()

タイトルの文字数の列を作る

In [ ]:
df['title_length'] = list(pd.Series(df['title_renamed']).apply(lambda x: len(x)))

df.head()

## データの可視化

蔵書数の確認

In [ ]:
df.shape[0]

年毎のデータの確認

In [ ]:
year_data.plot.bar()

月毎のデータの確認

In [ ]:
month_data.plot.bar()

日毎のデータの確認

In [ ]:
day_data.plot.bar()

曜日毎のデータの確認（0が月曜日、6が土曜日）

In [ ]:
weekday_data.plot.bar()

曜日毎のデータの確認（0が月曜日、6が日曜日）(全年)

In [ ]:
day_name_data.plot.bar()

曜日毎のデータの確認（0が月曜日、6が日曜日）(今年)

In [ ]:
weekday_data2.plot.bar()

曜日毎のデータの確認（曜日名で並べる）(全年)

In [ ]:
day_name_data.plot.bar()

曜日毎のデータの確認（曜日名で並べる）(今年)

In [ ]:
day_name_data2.plot.bar()

タイトルの文字数のヒストグラム

In [ ]:
df['title_length'].plot(kind='hist', bins=50, figsize=(16,4), alpha=0.5)

タイトル文字の分析

In [ ]:
df['title_length'].describe()

In [ ]:
title_max_len = df['title_length'].describe()['max']
title_min_len = df['title_length'].describe()['min']
title_mean_len = df['title_length'].describe()['50%']

行の最大文字数を設定（省略されるのを防止）

In [ ]:
pd.set_option("display.max_colwidth", 200)

タイトル文字数でタイトルを並び替え

In [ ]:
df.sort_values('title_length')['title_renamed']

3文字のタイトルを抽出

In [ ]:
df['title_renamed'][df['title_length'] == 3]

平均付近のタイトルを抽出

In [ ]:
df['title_renamed'][df['title_length'] == title_min_len]

In [ ]:
df['title_renamed'][df['title_length'] == title_max_len]

In [ ]:
df['title_renamed'][(df['title_length'] == title_mean_len)]

筆者の分析

In [ ]:
df['authors'].value_counts()

# 参考リンク

- https://qiita.com/ysdyt/items/9ccca82fc5b504e7913a
- https://qiita.com/canonno/items/57f917147b3d6ae106e9
- https://note.nkmk.me/python-pandas-multiple-conditions/
- https://note.com/nanigashi/n/n8a1e590d2ea3